## 1. Libraries and settings

In [ ]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
import random
import optuna

import torch
import torch.nn as nn
import torch.optim as optim


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight


import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt



In [ ]:


# Seed for Python's random module
random.seed(42)

# Seed for NumPy's random number generator
np.random.seed(42)

# Seed for PyTorch
torch.manual_seed(42)

# Additional settings for multi-threaded reproducibility
torch.use_deterministic_algorithms(True)  # This is a newer way compared to 'torch.backends.cudnn.deterministic'


In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)
df_filtered = df[df['signal'] != 'SignalNone']

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)

# # Assuming df is your DataFrame
# previous_signal = None  # Initialize a variable to keep track of the previous non-"SignalNone" value

# for i in range(len(df)):
#     if df.iloc[i, df_filtered.columns.get_loc('signal')] == "SignalNone" and previous_signal is not None:
#         df.iloc[i, df_filtered.columns.get_loc('signal')] = previous_signal  # Replace "SignalNone" with the previous signal
#     elif df.iloc[i, df_filtered.columns.get_loc('signal')] != "SignalNone":
#         previous_signal = df.iloc[i, df_filtered.columns.get_loc('signal')]  # Update the previous signal to the current one if it's not "SignalNone"

# df = df.loc[df['signal'] != 'SignalNone']

df['signal'] = df['signal'].replace({'SignalLong': 2, 'SignalShort': 0, 'SignalNone': 1})
df = df.ffill()

In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)


In [ ]:

hidden_dim = 32
num_layers = 2

num_epochs = 5
learning_rate=0.01
step_size=30
gamma=0.9

dropout_rate=0.2


In [ ]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test, indices_train_val, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(X_train_val, y_train_val, indices_train_val, test_size=0.2, shuffle=False)  # 0.2 here means 20% of the original data, or 25% of the training+validation set

# Now, `indices_val` holds the indices of your original dataset that were used for the validation set.


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

In [ ]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
timestep = 20

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_val_list = create_sequences(X_val_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_val_seq_ar = y_val[timestep:]
y_test_seq_ar = y_test[timestep:]

In [ ]:
x_train_array = np.array(X_train_list)
x_test_array = np.array(X_test_list)  

y_train_array = np.array(y_train_seq_ar).astype(np.int64)
y_test_array = np.array(y_test_seq_ar).astype(np.int64)

In [ ]:


# Move tensors to the MPS device
x_train_gpu = torch.from_numpy(x_train_array).type(torch.Tensor).to(device)
x_test_gpu = torch.from_numpy(x_test_array).type(torch.Tensor).to(device)
y_train_gpu = torch.from_numpy(y_train_array).long().to(device)
y_test_gpu = torch.from_numpy(y_test_array).long().to(device)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Assuming y_train is your target labels tensor for the training data
# and it's already in the form of a 1D tensor of class indices (0 to C-1)

# Convert y_train to a numpy array if it's a tensor
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = y_train  # Assuming y_train is already a numpy array

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_np), y=y_train_np)

# Convert class weights to a tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


input_dim = X_train_list.shape[2]  # Number of features
output_dim = 3  # Number of classes

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        # The input dimension is twice the hidden_dim because it's bidirectional
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Apply dropout to the output of the LSTM
        out = self.dropout(out)
        
        # Concatenate the hidden states from both directions
        out = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
        
        # Pass the concatenated hidden states to the fully connected layer
        out = self.fc(out)
        
        return out

# Create the model with bidirectional LSTM
model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim, dropout_rate=dropout_rate)
model.to(device)  # Move your model to the MPS device

# Loss function, optimizer, and scheduler remain the same
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=step_size, gamma=gamma)





In [ ]:
num_epochs = 10
print_epochs = 2
# hist will track the loss for now
hist = np.zeros(num_epochs)

# Ensure your model is in training mode
model.train()

for t in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    # Make sure x_train and y_train are already on the correct device (GPU)
    y_train_pred = model(x_train_gpu)

    # Compute loss
    loss = loss_fn(y_train_pred, y_train_gpu.long())  # Ensure y_train is of type torch.long
    if t % print_epochs == 0:  # Adjust logging frequency according to your preference
        print(f"Epoch {t}, Loss: {loss.item()}")
    hist[t] = loss.item()

    # Zero gradients before backward pass
    optimiser.zero_grad()

    # Perform backward pass: compute gradients of the loss with respect to all the learnable parameters
    loss.backward()

    # Update the parameters using the gradients and optimizer algorithm
    optimiser.step()
    
    # Step the scheduler
    scheduler.step()

    # Optional: Calculate and print accuracy or other metrics every few epochs
    if t % print_epochs == 0:  # Adjust as needed
        # Set the model to evaluation mode for accuracy calculation
        model.eval()
        with torch.no_grad():  # No need to track gradients for validation
            y_pred_tags = torch.argmax(torch.softmax(y_train_pred, dim=1), dim=1)
            correct_preds = (y_pred_tags == y_train_gpu).float().sum()
            accuracy = correct_preds / y_train_gpu.shape[0]
            print(f'Epoch {t} Accuracy: {accuracy.item() * 100:.2f}%')
        # Set the model back to training mode
        model.train()


In [ ]:
# # Adjust the figure size
# plt.figure(figsize=(6, 3))

# # Plot the training loss
# plt.plot(hist, label="Training loss")
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
with torch.no_grad():
    y_test_pred = model(x_test_gpu)
    # Convert logits to probabilities
    probabilities = torch.softmax(y_test_pred, dim=1)
    # Get the predicted class labels
    _, predicted_labels = torch.max(probabilities, 1)
    # Move the tensor to CPU and then convert to numpy
    predicted_labels_numpy = predicted_labels.cpu().numpy()
    print(len(predicted_labels_numpy))


In [ ]:
predicted_labels_numpy.shape

In [ ]:
df_split = data.data['symbol'][-len(predicted_labels_numpy):].copy()
df_split.loc[:, "signal"] = predicted_labels_numpy
signal = df_split['signal']
entries = signal == 2
exits = signal == 0

In [ ]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [ ]:
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 600
vbt.settings['plotting']['layout']['height'] = 300

In [ ]:
pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()

In [ ]:
pf.stats()

In [ ]:
stats = pf.stats()
total_return = stats['Total Return [%]']
orders = stats['Total Orders']
print("Total Orders:", orders)


In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# # Convert tensors to numpy arrays for use with Scikit-Learn
# true_labels = y_test.cpu().numpy()
# pred_labels = predicted_labels.cpu().numpy()

# precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for unweighted mean
# recall = recall_score(true_labels, pred_labels, average='macro')
# f1 = f1_score(true_labels, pred_labels, average='macro')
# conf_matrix = confusion_matrix(true_labels, pred_labels)

# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1 Score: {f1:.2f}')
# print('Confusion Matrix:\n', conf_matrix)